# Center-loaded plate

[![Google Collab Book](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/meyer-nils/torch-fem/blob/main/examples/basic/shell/plate.ipynb)

A square plate of 100x100 is fully clamped at all edges and subjected to a concentrated force F=16.367 at the center. A symmetric quarter model is computed. 


A reference solution is computed with ABAQUS for the following shell elements: 

| Type   | u_max  | 
|--------|--------| 
| S3     | -3.773 |
| STRI3  | -4.041 | 
| STRI65 | -4.486 | 
 

In [1]:
import meshzoo
import torch

from torchfem import Shell
from torchfem.materials import IsotropicElasticityPlaneStress

## Model setup

In [2]:
# Constants
L = 100.0
F = -16.367

# Material parameters (Cs is transverse shear stiffness)
mat = IsotropicElasticityPlaneStress(E=10000.0, nu=0.3)

# Define plate
X = torch.linspace(0.0, 0.5 * L, 9)
Y = torch.linspace(0.0, 0.5 * L, 9)
points, cells = meshzoo.rectangle_tri(X, Y, variant="zigzag")
N = len(points)
nodes = torch.hstack([torch.tensor(points, dtype=torch.float32), torch.zeros((N, 1))])
elements = torch.tensor(cells.tolist())

# Create Shell model
plate = Shell(nodes, elements, mat)

# Boundaries
top = nodes[:, 1] > (L / 2.0 - 0.01)
bottom = nodes[:, 1] < 0.01
left = nodes[:, 0] < 0.01
right = nodes[:, 0] > (L / 2.0 - 0.01)

# Force
plate.forces[0, 2] = F
plate.constraints[top] = True
plate.constraints[right] = True
plate.constraints[left, 0] = True
plate.constraints[left, 4] = True
plate.constraints[left, 5] = True
plate.constraints[bottom, 1] = True
plate.constraints[bottom, 3] = True
plate.constraints[bottom, 5] = True

## Solve 

In [ ]:
u, f = plate.solve()

# Print maximum displacement
print(f"u_max: {u.min():.2f}")

u_max: -3.99


## Visualize displacement 

The symmetric quarter model is mirrored in x-direction and y-direction.

In [4]:
plate.plot(
    u[:, 0:3],
    node_property={"u": torch.linalg.norm(u[:, 0:3], dim=1)},
    mirror=[True, True, False],
)

Widget(value='<iframe src="http://localhost:60067/index.html?ui=P_0x148f7ce20_0&reconnect=auto" class="pyvista…

## Visualize v. Mises stress

The symmetric quarter model is mirrored in x-direction and y-direction.

In [5]:
# Compute v. Mises stress
mises = plate.compute_stress(u, z=0.25, mises=True)

plate.plot(
    u[:, 0:3],
    element_property={"mises": mises},
    thickness=True,
    mirror=[True, True, False],
)

Widget(value='<iframe src="http://localhost:60067/index.html?ui=P_0x14c09ccd0_1&reconnect=auto" class="pyvista…